### Bibliothèques

In [10]:
import pandas as pd
import numpy as np
import random
import time
from scipy.stats import chi2_contingency

## FONCTIONS

### Fonction de Création d'une Clef de Répartition

In [11]:
# Tire une clef de repartition (en %) de manière aleatoire en fonction du nombre de classe voulue et d'une valeur planché de % de classe
# !!!! UN NOMBRE DE CLASSES TROP GRAND PEUT ETRE PARADOXAL AVEC LA VALEUR PLANCHé !!!! 
# Exemple : 10 classes avec une valeur planché est de 15% -> 15%*10 = 150% donc impossible
def clef_repartition_random(nb_classe,v_planche):
    clef_repartition = [v_planche] * nb_classe
    somme_restant = 100 - sum(clef_repartition)

    for i in range(nb_classe):
        if somme_restant <= 0:
            break
        max_val = somme_restant + clef_repartition[i]
        nouvelle_valeur = random.randint(v_planche, max_val)
        somme_restant -= (nouvelle_valeur - clef_repartition[i])
        clef_repartition[i] = nouvelle_valeur
    
    clef_repartition[-1] += somme_restant
    random.shuffle(clef_repartition)
    
    return clef_repartition

### Fonction de Répartition Cumulative des Valeurs

In [12]:
# Donne la repartition des individus en fonction d'une clef de repartition et la taille de l'échantillion
# Cette repartition est sous forme d'une liste donnant les index de bornes des différentes classes
def repartition_cumulative(clef_repartition, taille_serie) :
    repartition = [round(taille_serie * x / 100) for x in clef_repartition]

    # Avec les valeurs arrondies, on peut se retrouver avec des différences de +-1, on rectifie le resultat de manière arbitraire
    while sum(repartition) > taille_serie :
        repartition[-1] -= 1
    
    while sum(repartition) < taille_serie :
        repartition[-1] += 1

    cumulative_repartition = [sum(repartition[:i+1]) for i in range(len(repartition))]
    return cumulative_repartition

### Fonction donnant l'Encadrement des Classes

In [13]:
# Donne une liste avec les valeurs des encadements en fonction de la repartition cumulative et des valeurs de la serie
def encadrements_classe(repartition_cumulative, serie) :
    valeur = serie.iloc[[i - 1 for i in repartition_cumulative]].tolist()
    valeur = [min(serie)]+ valeur 
    return valeur

### Fonction de Subsitution des Valeurs de la Serie

In [14]:
# Remplace les valeurs quantitatives d'une serie par des classes selon un encadrement donné dans une liste
# Pas parfaitement optimisé car il transforme toute la série puis revient pour complter les NaN a cause de la valeur maximum qui n'est pas prit en compte cause de la borne exclue 

def substitutions_valeurs1(serie, bins):
    # Création des labels en fonction des intervalles
    labels = [f"[{bins[i]};{bins[i+1]}[" for i in range(len(bins)-2)]
    # Dernier intervalle inclut la borne droite
    labels.append(f"[{bins[-2]};{bins[-1]}]")  
    
    # pd.cut pour classer les valeurs dans les intervalles
    classes = pd.cut(serie, bins=bins, labels=labels, right=False)
    # Complete les valeurs manquantes (maximum de la serie) par la derniere borne
    classes = classes.fillna(f"[{bins[-2]};{bins[-1]}]")
    return classes

### FONCTION MAXI-BEST OF BIG MAC COCA

In [15]:
# On choisi 5 classes et un minimum de 10%
def generate_classes(nb_classe,v_planche, serie) :
    clef_repartition = clef_repartition_random(nb_classe,v_planche)
    repartition_cumul = repartition_cumulative(clef_repartition,len(serie))
    encadrement_classe = encadrements_classe(repartition_cumul, serie)
    serie_finale = substitutions_valeurs1(serie,encadrement_classe)
    return serie_finale

### Fonction Machine Learning

In [16]:
def optimize_classes(serie, df_income):
    """
    """
    def khi2(df_classes, df_income):
        """
        """
        table = pd.crosstab(df_classes, df_income)
        khi2, pval, ddl, contigent_theo = chi2_contingency(table)
        return khi2, pval, ddl
    
    df_classes_min = generate_classes(6,5, serie)
    khisq_min, pval_min, ddl_min = khi2(df_classes_min, df_income)
    n=100000
    deltas=[]

    for i in range(n):
        time1 = time.time()*1000
        df_classes = generate_classes(6,5, serie)
        time2 = time.time()*1000
        deltas.append((time2-time1)/1000)
        # print(i,'  Temps:', (deltas[-1]))
        khisq, pval, ddl = khi2(df_classes, df_income)
        if (khisq > khisq_min) and (pval<=pval_min):
            conv = i
            print(conv)
            df_classes_min = df_classes
            khisq_min = khisq
            pval_min = pval
            ddl_min = ddl
    sum_delta = sum(deltas)
    print('Temps total:', sum_delta, sum_delta/60)
    print('Temps moyen pour une génération de table:', sum_delta/n)
        
    return df_classes_min, khisq_min, pval_min, ddl_min, conv

## Test :

In [17]:
#df = pd.read_csv(r'files\clean.csv')

# Isolation de la variable age pour faire les tests
#df_income = df['income']
#serie = df['age']

# Ordonne la série dans l'ordre croissant et reinitialiser les index
#serie = serie.sort_values()
#serie = serie.reset_index(drop=True)

In [18]:
def main():
    df = pd.read_csv('files/clean.csv')
    df_income = df['income']

    serie = df['age']
    serie = serie.sort_values()
    serie = serie.reset_index(drop=True)

    df_classes, khisq, pval, ddl, conv = optimize_classes(serie, df_income)
    print("\n",df_classes, khisq, pval, ddl, conv)
    df_classes.to_csv(f'files/classes_opti_THEO{df_classes.name}.csv', index=False)


if __name__ == "__main__":
    main()

# passe la série en fichier csv
# serie_finale.to_csv('classes_random_age_THEO.csv', index=False)

# Met la serie original au coté de la serie finale pour vérifier les transformations effectuées
# df_test_comparaison = pd.DataFrame({'age_1': serie_finale, 'age_2': serie})
# df_test_comparaison.to_csv('classes_random_age_THEO.csv', index=False)

0
6
2464
8239
12161
43211
58408
Temps total: 490.70199536132617 8.178366589355436
Temps moyen pour une génération de table: 0.004907019953613262

 0        [17;28[
1        [17;28[
2        [17;28[
3        [17;28[
4        [17;28[
          ...   
45217    [51;90]
45218    [51;90]
45219    [51;90]
45220    [51;90]
45221    [51;90]
Name: age, Length: 45222, dtype: category
Categories (6, object): ['[17;28[' < '[28;30[' < '[30;32[' < '[32;34[' < '[34;51[' < '[51;90]'] 11.370309495845198 0.04451256462694012 5 58408
